# Preprocessing of Climate and Environment Data
This notebook loads raster data for climate and environmental data important to cropping production. Data is processed for an area of interest, and downscaled to a common spatial grid. Data are exported as a shape file and csv file. 

In [8]:
import pandas as pd
import geopandas as gpd 
from osgeo import gdal, ogr, osr, os
import numpy as np
from tobler.area_weighted import area_interpolate
import rioxarray as rxr
from shapely.geometry import mapping
from functools import reduce

import matplotlib.pyplot as plt

**Description of Raw Data**
1. Climate Data is obtained from the PRISM Climate Group.\
    a. 30-yr normals for the period of 1991-2020 \
    b. 800m resolution \
    c. Variables of mean annual temperature, mean total annual precipitation, and mean elevation which is used for a common grid reference \
    d. Data and metadata are availabe at https://prism.oregonstate.edu/normals/ 
2. Soil data is obtained from UC Davis compilation of SSURGO NRCS \
    a. SSURGO http://www.nrcs.usda.gov/wps/portal/nrcs/detail/soils/survey/?cid=nrcs142p2_053627 \
    b. 800m resolution \
    c. Variables of soil texture for 0-25 cm and 25 - 50 cm, plant available water content from 0-50 cm, and organic matter in kg/m2 \
    d. Data and metadata are available at https://casoilresource.lawr.ucdavis.edu/soil-properties/download.php
3. Climate Data for Potential Evapotranspiration (PET) is obtained from TERRACLIMATE \
    a. PET is provided as a 30-year monthly summary for the period of 1981-2010 as a compressed netCDF file \
    b. ~4 km resolution \
    b. PET is a derived data set and available at https://www.climatologylab.org/terraclimate.html 
    
**Description of Processed Data**
1. Clip raster data to Kansas bounds (function clips to a rectangle, data will later be clipped to irregular bounds), set to common crs, and convert to vectorized shapefiles 
2. Read in all processed shapefiles and interpolate polygons to the finest polygon level (elevation)
3. Merge all files on geometry
4. Clip to irregular Kansas bounds 
5. Save processed data to a CSV and a shape file 

**Coding reference**
1. EarthLab tutorial for raster to vector data transformations https://www.earthdatascience.org/courses/use-data-open-source-python/intro-vector-data-python/vector-data-processing/

**Read in each shapefile and clip to Kansas boundary**

In [9]:
# Kansas Shape file to clip data sources. Load data and select Kansas state boundaries  
fp= 'C:/Users/sarahann.USERS/Desktop/code/cb_2018_us_state_500k.shp' 

map_df = gpd.read_file(fp)
ks_map = map_df.NAME.isin(['Kansas']) 
ks_map = map_df[ks_map].loc[0:]
ks_map.to_crs(epsg=5070, inplace=True) 
ks_map.crs

<Derived Projected CRS: EPSG:5070>
Name: NAD83 / Conus Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - CONUS onshore - Alabama; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming.
- bounds: (-124.79, 24.41, -66.91, 49.38)
Coordinate Operation:
- name: Conus Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [14]:
# File paths for PRISM data: precipitation, temperature and elevation
precip="C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/PRISM_ppt_30yr_normal_800mM3_annual_bil.bil"
temp="C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/PRISM_tmean_30yr_normal_800mM3_annual_bil.bil"
elv="C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/PRISM_us_dem_800m_bil.bil"
elv_4km="C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/PRISM_us_dem_4km_bil.bil"

# File path for soil data
paws_050 = "C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/paws_050.tif"
om_kg_sq_m = "C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/om_kg_sq_m.tif"
sand= "C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/sand.tif"
silt= "C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/silt.tif"
clay= "C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/clay.tif"
depth= "C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/soil_depth.tif"
depth_restriction="C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/resdept.tif"

In [15]:
# Open all files, clip to Kansas Boundaries, and then save as a new tif 
# Note, this clip will clip to the farthest parameter edge. The final data, once interpolated will be clipped with geopandas to get the exact boundaries of Kansas. This is computationally expensive and is only done on the final output layer.

files = [precip, temp, elv, elv_4km, paws_050, om_kg_sq_m, sand, silt, clay, depth, depth_restriction] 
files_str = ['precip', 'temp', 'elv', 'elv_4km','paws_050', 'om_kg_sq_m', 'sand', 'silt', 'clay', 'depth', 'dep_res']

for i,j in zip(files, files_str):
    x = rxr.open_rasterio(i).squeeze() # .squeeze removes a third dimension of 1 for band  and maintains a 2 dimension object
    x = x.rio.reproject(ks_map.crs)
    x=x.rio.clip(ks_map.geometry.apply(mapping),
                                          # This is needed if your df is in a diff CRS than the raster data, which is the case for the soil data in 5070
                                          ks_map.crs)
    x.rio.to_raster('C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/'+j+'_ks_clip.tif')

**Read each clipped shape tif (raster) and convert and save as a shape file (vector)**

In [16]:
# File paths for PRISM data: precipitation, temperature and elevation clipped to Kansas 
precip="C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/precip_ks_clip.tif"
temp="C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/temp_ks_clip.tif"
elv="C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/elv_ks_clip.tif"
elv_4km="C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/elv_4km_ks_clip.tif"

# File path for soil data clipped to Kansas 
paws_050 = "C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/paws_050_ks_clip.tif"
om_kg_sq_m = "C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/om_kg_sq_m_ks_clip.tif"
sand = "C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/sand_ks_clip.tif"
silt = "C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/silt_ks_clip.tif"
clay = "C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/clay_ks_clip.tif"
depth = "C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/depth_ks_clip.tif"
depth_res = "C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/depth_rest_ks_clip.tif"
pet = "C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/pet_ks.tif" # This file is generated from an nc file. File was processed and clipped to Kansas seperately with all months added for an annual sum of PET

In [124]:
# Function to open each file, and save to a vector data format/shape file
def vec2rast (path, string):
    prism_ds = gdal.Open(path)                      
    band = prism_ds.GetRasterBand(1)
    dst_layername = string
    drv = ogr.GetDriverByName('ESRI Shapefile')

    # customer out path, and file name
    outfile = drv.CreateDataSource(r'C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/'+ string +'.shp') 
    outlayer = outfile.CreateLayer('polygonized raster', srs = None )  
    newField = ogr.FieldDefn(string, ogr.OFTReal)            #custom variable name
    outlayer.CreateField(newField)

    gdal.Polygonize(band, None, outlayer, 0, [])
    outfile = None

In [40]:
# Run Function for each raster and save as a shape file
vec2rast(elv,'elevation')
vec2rast(elv_4km,'elevation_4km')
vec2rast(temp,'temp')
vec2rast(precip,'precip')
vec2rast(paws_050,'paws_050') 
vec2rast(om_kg_sq_m, 'om_kg_sq_m')
vec2rast(sand, 'sand')
vec2rast(silt, 'silt')
vec2rast(clay, 'clay')
vec2rast(depth, 'depth')
vec2rast(depth_res, 'depth_res')
vec2rast(pet, 'pet')

**Read each shape file (vector), and downscale to common geometry polygons**

In [133]:
# Function to open each file, read as a geopandas and set to commmon crs
variable_list = ['elevation', 'temp','precip','paws_050','om_kg_sq_m', 'sand', 'silt', 'clay', 'depth', 'depth_rest']
                                                                                                    
ks_variable = []

for string in variable_list:
    x = gpd.read_file('C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/'+string+'.shp')
    x.set_crs(epsg=5070, inplace=True) # using a metric unit and not a degree unit for geopandas calculations. 
    x.to_crs(epsg=5070)  
    ks_variable.append(x) 

In [178]:
# PET tif does not have geometry column set in the geopandas so will not convert geometries to crs. Here geometry is set and the crs subsequently set. 
x = gpd.read_file('C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/pet.shp')
x.set_crs(epsg=4326, inplace=True) # using a metric unit and not a degree unit for geopandas calculations.
x.set_geometry("geometry")
x.to_crs(epsg=5070, inplace=True)  
ks_variable.append(x) 

In [183]:
# Determine the finest geometrical grid by printing geometry count for each geopandas 
for i in ks_variable:
    print(i.geometry.count())

256680
463
100212
49335
205374
215481
214330
182929
193586
158972
10440


In [184]:
print('The Variables in order of the list are:')
for i in ks_variable:
    print([i.columns[0]])

The Variables in order of the list are:
['elevation']
['temp']
['precip']
['paws_050']
['om_kg_sq_m']
['sand']
['silt']
['clay']
['depth']
['depth_rest']
['pet']


In [263]:
# 0 is PET Nan value, change to -999999
ks_variable[10]['pet'].mask(ks_variable[10]['pet'] == 0, -999999.0, inplace=True) 

In [272]:
# Check for Nan values, and validate there are no other - values
print('The number of NaNs or -9999 for any variable is:')
for i in ks_variable:
    x = i.columns[0]
    count = i[x].value_counts()[-9999.0]
    less_than = i[i.columns[0]][i[i.columns[0]] <= 0].count()
    print(x, count, less_than) 

The number of NaNs or -9999 for any variable is:
elevation 4 4
temp 4 4
precip 4 4
paws_050 53 53
om_kg_sq_m 53 53
sand 53 53
silt 53 53
clay 53 53
depth 53 53
depth_rest 2339 2344
pet 2 2


**Interpolation with Tobler**
1. The variables of precipitation, temperature, PAWS, and organic matter are interpolated as extensive values, under the assumption that only areas for intersected features are utilized. 
2. The variables of texture for 0-25cm, and 25-50cm are calculated as categorical variables with final value selected on the high percent represented value in the polygon. 
3. Documentation for Tobler https://pysal.org/tobler/generated/tobler.area_weighted.area_interpolate.html#tobler.area_weighted.area_interpolate
4. A great explaination on interpolation methods is found here https://slu-opengis.github.io/areal/articles/areal-weighted-interpolation.html#extensive-and-intensive-interpolations-1

In [314]:
# Interpolate every variable to the same geometry as the elevation file (266147 unique polygons)
ks_variable_interp= []

for i in range(len(ks_variable)): # the first geo df is elevation and has the finest scale of geometries, so range count will start at 1 instead of 0 
    x = area_interpolate(source_df=ks_variable[i],target_df=ks_variable[0], intensive_variables=[ks_variable[i].columns[0]]) 
    ks_variable_interp.append(x)

In [345]:
#Merge all files in the data frame
merge_df=reduce(lambda x, y: pd.merge(x, y, on = 'geometry'), ks_variable_interp)

In [346]:
#Check crs before clipping to Kansas Boundaries
merge_df.crs == ks_map.crs

True

**Clip to irregular Kansas bounds**

In [347]:
ks_variables=gpd.overlay(merge_df, ks_map,  how='intersection')  

In [355]:
# Drop columns from ks_map
ks_variables=ks_variables.drop(columns=['STATEFP', 'STATENS', 'AFFGEOID', 'GEOID', 'STUSPS', 'NAME', 'LSAD','ALAND', 'AWATER'])

In [356]:
ks_variables.describe()

,elevation,temp,precip,paws_050,om_kg_sq_m,sand,silt,clay,depth,depth_rest,pet
count,256680.00,256680.00,256680.00,256680.00,256680.00,256680.00,256680.00,256680.00,256680.00,256680.00,256680.00
mean,575.41,12.48,748.50,-9.99,1.52,2.31,31.24,10.15,165.56,-2613.87,1265.00
std,252.30,39.53,206.26,313.38,313.83,314.28,314.94,314.19,319.52,4223.60,428.44
min,-9999.00,-9999.00,-9999.00,-9999.00,-9999.00,-9999.00,-9999.00,-9999.00,-9999.00,-9999.00,-9999.00
25%,376.00,12.00,568.00,7.95,17.00,9.03,46.42,22.63,174.60,-6275.30,1193.00
50%,492.00,13.00,729.00,8.98,19.70,14.84,52.18,27.93,193.70,62.88,1272.00
75%,763.00,13.00,918.00,10.00,23.61,23.49,59.26,36.39,199.36,80.02,1375.00
max,1226.00,15.00,1206.98,13.00,69.85,97.00,75.67,60.28,235.07,208.00,1590.00


In [357]:
# Do any of the columns have nan values
ks_variables.loc[ks_variables['precip'] < 0]

,elevation,temp,precip,paws_050,om_kg_sq_m,sand,silt,clay,depth,depth_rest,pet,geometry
221499,-9999.00,-9999.00,-9999.00,-9878.63,-9878.50,-9878.50,-9878.09,-9878.43,-9876.53,-9903.89,-9651.42,"MULTIPOLYGON (((120879.226 1593582.536, 120879..."
245032,-9999.00,-9999.00,-9999.00,-9881.86,-9881.78,-9881.70,-9881.34,-9881.70,-9879.84,-9987.75,1516.48,"MULTIPOLYGON (((-511701.775 1898603.778, -5120..."
253561,-9999.00,-9999.00,-9999.00,-9801.15,-9801.00,-9800.33,-9800.68,-9800.88,-9797.63,-9869.09,1485.01,"MULTIPOLYGON (((-90982.820 1551510.640, -90982..."
256679,-9999.00,-9999.00,-9999.00,-7658.99,-7655.23,-7657.95,-7649.21,-7653.24,-7615.78,-7872.60,1221.12,"POLYGON ((121630.510 1551510.640, 121630.510 1..."


In [359]:
# We will drop these four roads
idx = ks_variables[ks_variables['precip'] < 0].index
ks_variables.drop(idx, inplace=True)
print(ks_variables.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 256676 entries, 0 to 256678
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   elevation   256676 non-null  float64 
 1   temp        256676 non-null  float64 
 2   precip      256676 non-null  float64 
 3   paws_050    256676 non-null  float64 
 4   om_kg_sq_m  256676 non-null  float64 
 5   sand        256676 non-null  float64 
 6   silt        256676 non-null  float64 
 7   clay        256676 non-null  float64 
 8   depth       256676 non-null  float64 
 9   depth_rest  256676 non-null  float64 
 10  pet         256676 non-null  float64 
 11  geometry    256676 non-null  geometry
dtypes: float64(11), geometry(1)
memory usage: 25.5 MB
None


In [362]:
# Drop the nan rows for soil, PAWS_050 has the greatest number of NaN values for soil data, with the exception of depth_rest
idx = ks_variables[ks_variables['paws_050'] < 0].index
ks_variables.drop(idx, inplace=True)
print(ks_variables.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 255302 entries, 1 to 256678
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   elevation   255302 non-null  float64 
 1   temp        255302 non-null  float64 
 2   precip      255302 non-null  float64 
 3   paws_050    255302 non-null  float64 
 4   om_kg_sq_m  255302 non-null  float64 
 5   sand        255302 non-null  float64 
 6   silt        255302 non-null  float64 
 7   clay        255302 non-null  float64 
 8   depth       255302 non-null  float64 
 9   depth_rest  255302 non-null  float64 
 10  pet         255302 non-null  float64 
 11  geometry    255302 non-null  geometry
dtypes: float64(11), geometry(1)
memory usage: 25.3 MB
None


In [374]:
idx = ks_variables[ks_variables['pet'] < 180].index
ks_variables.drop(idx, inplace=True)
print(ks_variables.info())
ks_variables.describe()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 254942 entries, 1 to 256678
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   elevation   254942 non-null  float64 
 1   temp        254942 non-null  float64 
 2   precip      254942 non-null  float64 
 3   paws_050    254942 non-null  float64 
 4   om_kg_sq_m  254942 non-null  float64 
 5   sand        254942 non-null  float64 
 6   silt        254942 non-null  float64 
 7   clay        254942 non-null  float64 
 8   depth       254942 non-null  float64 
 9   depth_rest  254942 non-null  float64 
 10  pet         254942 non-null  float64 
 11  geometry    254942 non-null  geometry
dtypes: float64(11), geometry(1)
memory usage: 25.3 MB
None


,elevation,temp,precip,paws_050,om_kg_sq_m,sand,silt,clay,depth,depth_rest,pet
count,254942.00,254942.00,254942.00,254942.00,254942.00,254942.00,254942.00,254942.00,254942.00,254942.00,254942.00
mean,576.02,12.64,748.19,8.73,20.26,21.06,50.03,28.90,184.63,-2598.24,1283.00
std,248.31,0.98,201.47,1.39,6.00,18.33,13.30,8.89,19.92,4221.98,113.65
min,214.00,10.99,429.00,3.00,1.00,2.11,1.00,2.00,42.00,-9999.00,312.52
25%,377.00,12.00,568.00,7.97,17.05,9.12,46.63,22.71,175.08,-6171.14,1193.70
50%,493.00,13.00,728.00,8.99,19.73,14.93,52.22,27.95,193.84,63.00,1272.00
75%,764.00,13.00,917.00,10.00,23.63,23.60,59.31,36.43,199.38,80.09,1375.00
max,1226.00,15.00,1205.50,13.00,69.85,97.00,75.67,60.28,235.07,208.00,1590.00


In [377]:
# Set all depth_rest values to NaN. These areas do not have a restrictive layer and thus we will retain the full depth value
ks_variables['depth_rest'].mask(ks_variables['depth_rest'] < 0, np.NaN, inplace=True) # convert all nan values nan for restrictive depth before masking in the next line

ks_variables['depth_adj']=ks_variables['depth'].mask(ks_variables['depth_rest'] < ks_variables['depth'], ks_variables['depth_rest']) # set values to restrictive depth

ks_variables['depth_adj']=ks_variables['depth_adj'].mask(ks_variables['depth_adj'] > 150, 150) # set all values to no higher than 150 cm depth (max effective root zone)
ks_variables.describe()

,elevation,temp,precip,paws_050,om_kg_sq_m,sand,silt,clay,depth,depth_rest,pet,depth_adj
count,254942.00,254942.00,254942.00,254942.00,254942.00,254942.00,254942.00,254942.00,254942.00,169926.00,254942.00,254942.00
mean,576.02,12.64,748.19,8.73,20.26,21.06,50.03,28.90,184.63,78.63,1283.00,102.24
std,248.31,0.98,201.47,1.39,6.00,18.33,13.30,8.89,19.92,25.74,113.65,39.40
min,214.00,10.99,429.00,3.00,1.00,2.11,1.00,2.00,42.00,0.02,312.52,0.02
25%,377.00,12.00,568.00,7.97,17.05,9.12,46.63,22.71,175.08,63.00,1193.70,69.96
50%,493.00,13.00,728.00,8.99,19.73,14.93,52.22,27.95,193.84,76.29,1272.00,89.00
75%,764.00,13.00,917.00,10.00,23.63,23.60,59.31,36.43,199.38,89.00,1375.00,150.00
max,1226.00,15.00,1205.50,13.00,69.85,97.00,75.67,60.28,235.07,208.00,1590.00,150.00


In [388]:
ks_variables=ks_variables.drop(columns=['depth_rest', 'depth'])

**Save processed data to a CSV and a shape file.**

In [390]:
#Export Data as a CSV and shape files
ks_variables.to_csv('ks_variables.csv')
ks_variables.to_file('ks_variables.shape')